##  Importing numpy package for vectorized implementation

In [2]:
import numpy as np

## Importing Matplot package for ploting cost vs number of iteration graph (graph is optional)

In [3]:
import matplotlib.pyplot as plt

## Initializing Weight and bias units

In [5]:
def initialize_parameters_deep(layer_dims):
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros(shape=(layer_dims[l],1))
        
    return parameters

## Function for Forward Propagation in Neural Network

In [6]:
def linear_activation_forward(A_prev, W, b, activation):
    
    Z = np.dot(W,A_prev) + b
    linear_cache = (A_prev, W, b)
    activation_cache = Z
    
    if activation == "sigmoid":
        A = 1/(1+np.exp(-Z))
        
    elif activation == "relu":
        A = np.maximum(0,Z)
    
    cache = (linear_cache, activation_cache)

    return A, cache


def L_model_forward(X, parameters):
    
    np.random.seed(1)
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    return AL, caches

## Function to Compute the cost of model 

In [7]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = (-1 / m) * (np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL))))
    cost = np.squeeze(cost)
    
    return cost

## Function for backward propagation in Neural network (use of gradient descent for optimizing parameters)

In [8]:
def relu_backward(dA, cache):
    
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoid_backward(dA, cache):
    
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    return dZ

def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
                
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    
    np.random.seed(1)
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

## Function for changing parameters after each optimization

In [9]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l + 1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= learning_rate * grads["db" + str(l + 1)]
    
    return parameters

## Function that builds the whole Neural Network by the functions implemented previously

In [10]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 1000, print_cost=False):
    
    costs = []                         # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):   # Loop (gradient descent)

        AL, caches = L_model_forward(X, parameters)
        
        if print_cost:
            cost = compute_cost(AL, Y)
        
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    
    if print_cost :
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per hundreds)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
    
    return parameters

## Prediction on Test Dataset 

In [12]:
def predict(X, y, parameters,print_accuracy=False):
    
    # Forward propagation
    AL, caches = L_model_forward(X, parameters)
    predictions = np.round(AL)
    
    if print_accuracy:
        m = X.shape[1]
        n = len(parameters) // 2
        p = np.zeros((1,m))
        print("Accuracy: "  + str(np.sum((predictions == y)/m)))
        
    return predictions

## Initiliazing the size of neural network

In [13]:
layers_dims = [2, 20, 7, 5, 1] #  4-layer model

## layers_dims[0] - input layer (Number of features)
## layers_dims[1...3] - hidden layer
## layers_dims[4] - output layer (1 unit for binomial classification)